In [1]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split
from catboost import Pool, cv
random_state = 7
import time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import os
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
import warnings
import numpy as np
warnings.filterwarnings("ignore")

In [2]:
def model_validation(tr,ts, iterations, learning_rate,depth, class_weight, n_splits = 10):
   
    # get current directory
    path = os.getcwd()
    parent = os.path.dirname(path)

    train = pd.read_csv(parent + '\Data\\' + tr + ".csv")
    test = pd.read_csv(parent + '\Data\\' + ts + ".csv")
    
    X_train = train.drop("target", axis=1)
    y_train = train["target"]
    
    X_test  = test.drop("target", axis=1)
    y_test = test["target"]
    
        # initialize Pool
    train_pool = Pool(X_train, 
                      y_train)
        # initialize Pool
    test_pool = Pool(X_test, 
                      y_test)
    
    scores = pd.DataFrame(columns=["Iterations", "Learning Rate", "Depth", "F1", "ROC", "Sensitivity", "Specificty", "ACC" ,  "CV F1", "CV ROC", "ROC Sens", "ROC Spec", "CV ACC"])

    for iteration in iterations:
        for rate in learning_rate:
            for deep in depth:
                    start = time.time()
                    model = CatBoostClassifier(iterations=iteration,
                           depth=deep,
                           learning_rate=rate,
                           class_weights=class_weight,
                           loss_function='Logloss',
                           random_state = random_state,
                           verbose=False)
                    
                    model.fit(train_pool)
                    pred = model.predict(X_test)
                    f1 = (f1_score(pred, y_test))
                    
                    cm = confusion_matrix(y_test, pred)
                    total=sum(sum(cm))
                    roca = round(roc_auc_score(y_test, pred),4)
                    sens = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                    spec = round(cm[0,0]/(cm[1,0]+cm[0,0]),4)                    
                    acc = round((cm[1,1] + cm[0,0])/(cm[1,1] + cm[0,0] + cm[0,1] + cm[0,1]))
                    
                    
                    scores_cv = []
                    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=random_state)
                    skf.get_n_splits(X_train, y_train)
                    for train_index, test_index in skf.split(X_train,y_train):
                        X_cv_train, X_cv_test = X_train.iloc[train_index], X_train.iloc[test_index]
                        y_cv_train, y_cv_test = y_train.iloc[train_index], y_train.iloc[test_index]

                        model_cv = CatBoostClassifier(iterations=iteration,
                                               depth=deep,
                                               learning_rate=rate,
                                               class_weights=class_weight,
                                               loss_function='Logloss',
                                               random_state = random_state,
                                               verbose=False)

                        # initialize Pool
                        train_pool = Pool(X_cv_train, 
                                          y_cv_train)
                            # initialize Pool
                        test_pool = Pool(X_cv_test, 
                                          y_cv_test)

                        model_cv.fit(train_pool)
                        pred = model_cv.predict(X_cv_test)
                        f1_cv = (f1_score(pred, y_cv_test))

                        cm = confusion_matrix(y_cv_test, pred)
                        total=sum(sum(cm))
                        roca_cv = round(roc_auc_score(y_cv_test, pred),4)
                        sens_cv = round(cm[1,1]/(cm[1,1]+cm[0,1]),4)
                        spec_cv = round(cm[0,0]/(cm[1,0]+cm[0,0]),4) 
                        acc_cv = round((cm[1,1] + cm[0,0])/(cm[1,1] + cm[0,0] + cm[0,1] + cm[0,1]))
                        scores_cv.append([f1_cv, roca_cv, sens_cv, spec_cv, acc_cv])

                      
                    
                    mean = (np.mean(np.array(scores_cv), axis=0))
                    df_new_row = pd.DataFrame(data=([[iteration, rate, deep, f1, roca, sens, spec, acc,mean[0], mean[1], mean[2], mean[3], mean[4]]]), 
                                              columns=["Iterations", "Learning Rate", "Depth",  "F1", "ROC", "Sensitivity", "Specificty", "ACC", 
                                                      "CV F1", "CV ROC", "ROC Sens", "ROC Spec", "CV ACC"])
                    scores = pd.concat([scores,df_new_row], ignore_index=True)
                    end = time.time()
                    print(end - start)
                    
            
    return scores

In [3]:
# tr = "trainset_ALLvsHealthy_80_target_in"
# ts = "testset_ALLvsHealthy_20_target_in"

# tr = "trainset_AMLvsALL_90_target_in"
# ts = "testset_AMLvsALL_10_target_in"

tr = "trainset_breast_healthy_80_target_in"
ts = "testset_breast_healthy_20_target_in"



# start = time.time()

#  iterations, learning_rate,depth, weights, n_splits = 10
results = model_validation(tr,ts, [100, 200, 300], [0.1,0.2, 0.3, 0.4],[3,4,5], [0.12 , 1])


160.73616886138916
272.4365038871765
438.61100125312805
166.5114996433258
285.9999997615814
462.39950156211853
169.95549941062927
283.0034999847412
510.77749967575073
182.13950109481812
289.6029963493347
485.7279996871948
346.64649629592896
579.0245013237
995.1725010871887
358.30549931526184
470.0775022506714
787.9575023651123
289.5079960823059
467.39600014686584
781.2400045394897
284.9920003414154
470.9800012111664
782.8239977359772
418.5190043449402
684.5544974803925
1154.9754974842072
419.27599906921387
685.5894999504089
1160.8480017185211
417.95550203323364
684.9090044498444
1158.9099950790405
416.63400077819824
682.6890006065369
1156.6592559814453


In [16]:
index = 0
top = 0
i = 0
for row in results.loc[:]["CV F1"]:
    if row > top:
        top = row
        index = i
    i = i + 1
print(results.loc[index])
for item in results.loc[index]:
    print(item)

Iterations            100
Learning Rate         0.3
Depth                   3
F1               0.933333
ROC                0.9718
Sensitivity         0.913
Specificty         0.9945
ACC                     1
CV F1            0.930417
CV ROC            0.97573
ROC Sens          0.90571
ROC Spec          0.99593
CV ACC                  1
Name: 6, dtype: object
100
0.3
3
0.9333333333333332
0.9718
0.913
0.9945
1.0
0.9304167280947466
0.9757299999999999
0.90571
0.9959299999999999
1.0
